# Apartment Price Prediction - Final Model

## 1. Introduction
Goal: Predict the offering price of Prague apartments.
Metric: MAPE (Mean Absolute Percentage Error)

This notebook implements the final solution. It includes:
- **Advanced Feature Engineering**: Distance to center, layout parsing, text mining, clustering.
- **Hyperparameter Tuning**: Code included but commented out to save time.
- **Stacking Ensemble**: Combining XGBoost, LightGBM, and CatBoost.
- **Final Training**: Trained on the **entire** training dataset.

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.impute import SimpleImputer
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import re

%matplotlib inline

## 2. Data Loading & Feature Engineering

In [29]:
# Haversine Distance Function
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Load Data
train_df = pd.read_csv('appartments_train.csv')
test_df = pd.read_csv('appartments_test.csv')

# Separate target and log-transform it
X = train_df.drop(columns=['price'])
y = train_df['price']
y_log = np.log1p(y)

X_test = test_df.copy()
if 'price' in X_test.columns:
    X_test = X_test.drop(columns=['price'])

# Combine
combined = pd.concat([X, X_test], axis=0).reset_index(drop=True)

print("Feature Engineering...")

# 1. Distance to Center (Wenceslas Square: 50.0812, 14.4280)
combined['dist_center'] = haversine_distance(combined['gps_lat'], combined['gps_lon'], 50.0812, 14.4280)

# 2. Layout Parsing
def parse_layout(layout):
    if pd.isna(layout): return 0, 0
    rooms = re.search(r'(\d+)', str(layout))
    rooms = int(rooms.group(1)) if rooms else 1
    kk = 1 if 'kk' in str(layout).lower() else 0
    return rooms, kk

combined[['n_rooms', 'has_kk']] = combined['layout'].apply(lambda x: pd.Series(parse_layout(x)))

# 3. Date Features
combined['first_seen'] = pd.to_datetime(combined['first_seen'])
combined['last_seen'] = pd.to_datetime(combined['last_seen'])
min_date = combined['first_seen'].min()
combined['days_since_first_seen'] = (combined['first_seen'] - min_date).dt.days
combined['days_on_market'] = (combined['last_seen'] - combined['first_seen']).dt.days

# 4. Text Mining (TF-IDF + Regex)
combined['text'] = combined['text'].fillna('').astype(str).str.lower()
keywords = {
    'luxus': r'luxus|nadstandard',
    'rekonstrukce': r'rekonstrukc|zrekonstru',
    'novostavba': r'novostavb|projekt',
    'metro': r'metro',
    'park': r'park',
    'balkon': r'balkon|lodži|terasa',
    'sklep': r'sklep|komora',
    'garaz': r'garáž|parkování|stání',
    'cihla': r'cihl'
}
for key, pattern in keywords.items():
    combined[f'has_{key}'] = combined['text'].str.contains(pattern, regex=True).astype(int)

tfidf = TfidfVectorizer(max_features=200, stop_words='english', ngram_range=(1, 2))
text_features = tfidf.fit_transform(combined['text'])
svd = TruncatedSVD(n_components=30, random_state=42)
text_pca = svd.fit_transform(text_features)
text_df = pd.DataFrame(text_pca, columns=[f'text_pca_{i}' for i in range(30)])
combined = pd.concat([combined, text_df], axis=1)

# 5. Geospatial Clustering
coords = combined[['gps_lat', 'gps_lon']].fillna(combined[['gps_lat', 'gps_lon']].mean())
kmeans = KMeans(n_clusters=50, random_state=42, n_init=10)
combined['loc_cluster'] = kmeans.fit_predict(coords)

# 6. Basic Cleaning
fill_zero_cols = ['cellar_area', 'balcony_area', 'garden_area', 'parking']
for col in fill_zero_cols:
    combined[col] = combined[col].fillna(0)
poi_nearest_cols = [c for c in combined.columns if 'nearest' in c]
for col in poi_nearest_cols:
    combined[col] = combined[col].fillna(combined[col].max() * 2.0)
combined['elevator'] = combined['elevator'].fillna('Unknown')

# 7. Ratios
combined['floor_ratio'] = combined['floor'] / combined['total_floors']
combined['floor_ratio'] = combined['floor_ratio'].fillna(0)
combined['total_area'] = combined['area'] + combined['cellar_area'] + combined['balcony_area'] + combined['garden_area']

# Drop columns
drop_cols = ['id', 'text', 'address', 'first_seen', 'last_seen']
combined = combined.drop(columns=drop_cols)

# Encode Categorical
cat_cols = ['layout', 'construction', 'condition', 'ownership', 'elevator', 'loc_cluster']
combined = pd.get_dummies(combined, columns=cat_cols, drop_first=True)

# Split back
X_processed = combined.iloc[:len(X)].copy()
X_test_processed = combined.iloc[len(X):].copy()

# Global Imputation
imputer = SimpleImputer(strategy='median')
X_processed_imputed = imputer.fit_transform(X_processed)
X_test_processed_imputed = imputer.transform(X_test_processed)
X_processed = pd.DataFrame(X_processed_imputed, columns=X_processed.columns)
X_test_processed = pd.DataFrame(X_test_processed_imputed, columns=X_test_processed.columns)

print(f"Processed shape: {X_processed.shape}")

Feature Engineering...
Processed shape: (5000, 138)


## 3. Hyperparameter Tuning (Grid Search)
**Note:** This section is commented out because it takes a very long time to run. The parameters used in the final model were derived from previous runs of this tuning process.

In [30]:
# --- Example Grid Search for XGBoost ---
# xgb_param_dist = {
#     'n_estimators': [1000, 3000, 5000],
#     'learning_rate': [0.005, 0.01, 0.05],
#     'max_depth': [6, 8, 10],
#     'subsample': [0.6, 0.8],
#     'colsample_bytree': [0.6, 0.8]
# }
# xgb_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)
# xgb_search = RandomizedSearchCV(xgb_model, xgb_param_dist, n_iter=10, scoring='neg_mean_absolute_percentage_error', cv=3, random_state=42, n_jobs=-1)
# xgb_search.fit(X_processed, y_log)
# print("Best XGB Params:", xgb_search.best_params_)

# Similar blocks can be added for LightGBM and CatBoost

## 4. Stacking Ensemble & Cross-Validation
**Note:** The Cross-Validation loop is commented out to speed up the notebook execution. It was used to verify the model performance (approx. 9.35% MAPE).

In [31]:
# Base Models (Optimized Parameters)
xgb_model = xgb.XGBRegressor(
    n_estimators=5000, learning_rate=0.005, max_depth=8, 
    subsample=0.6, colsample_bytree=0.6, random_state=42, n_jobs=-1,
    reg_alpha=0.1, reg_lambda=0.1
)

lgb_model = lgb.LGBMRegressor(
    n_estimators=5000, learning_rate=0.005, num_leaves=80, 
    subsample=0.6, colsample_bytree=0.6, random_state=42, n_jobs=-1, verbose=-1,
    reg_alpha=0.1, reg_lambda=0.1
)

cb_model = cb.CatBoostRegressor(
    iterations=5000, learning_rate=0.005, depth=9, l2_leaf_reg=5,
    random_state=42, verbose=False, allow_writing_files=False,
    bagging_temperature=0.2
)

# Meta Learner
meta_learner = RidgeCV()

estimators = [
    ('xgb', xgb_model),
    ('lgb', lgb_model),
    ('cb', cb_model)
]

stacking_reg = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_learner,
    cv=5,
    n_jobs=-1,
    passthrough=True
)

# --- Cross-Validation (Commented Out) ---
# print("Evaluating with 5-Fold CV...")
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# mape_scores = []
# for fold, (train_idx, val_idx) in enumerate(kf.split(X_processed, y_log)):
#     X_tr, X_val = X_processed.iloc[train_idx], X_processed.iloc[val_idx]
#     y_tr, y_val = y_log.iloc[train_idx], y_log.iloc[val_idx]
#     stacking_reg.fit(X_tr, y_tr)
#     y_pred_log = stacking_reg.predict(X_val)
#     y_pred = np.expm1(y_pred_log)
#     y_true = np.expm1(y_val)
#     score = mean_absolute_percentage_error(y_true, y_pred)
#     mape_scores.append(score)
#     print(f"Fold {fold+1} MAPE: {score:.4f}")
# print(f"Average MAPE: {np.mean(mape_scores):.4f}")

## 5. Final Prediction
Training the model on the **full training dataset** and generating predictions.

In [32]:
print("Retraining on full data...")
stacking_reg.fit(X_processed, y_log)
final_preds_log = stacking_reg.predict(X_test_processed)
final_preds = np.expm1(final_preds_log)

submission = pd.DataFrame({
    'id': test_df['id'],
    'price': final_preds
})
submission.to_csv('Data_nerds_predikce.csv', index=False)
print("Submission saved to Data_nerds_predikce.csv")

Retraining on full data...
Submission saved to Data_nerds_predikce.csv


In [33]:
submission.head()

,id,price
0,8795,7.509256e+06
1,6516,8.615630e+06
2,4714,5.248091e+06
3,8423,7.600472e+06
4,5361,7.535799e+06
